***Project Preview:***
* Aim : In this study we aim to reveal whether there is a significant difference between the 3 KPI for two versoins of a game platform. 

> For furthur reading read [this post](https://www.kaggle.com/babyoda/a-b-testing-in-practice/notebook).  


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns
import warnings
import numpy as np
from scipy.stats import shapiro
%matplotlib inline
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# **Reading the file and exploring the data**

In [ ]:
df = pd.read_csv("../input/mobile-games-ab-testing/cookie_cats.csv")
print(df.head())



### The Key Performance metrics: 
* Retention_1 -  the player come back and play 1 day after installing
* Retention_7 -  the player come back and play 7 days after installing
* sum_gamerounds - The rounds one player plays the game


In [ ]:
des = df.groupby("version").sum_gamerounds.agg(["count", "median", "mean", "std", "max"])
print(des)

In [ ]:
## the variance in one group is much higher, so we check the data with a scatter plot
df.plot.scatter(x='version', y='sum_gamerounds', title= "Scatter plot between two variables X and Y")
plt.show()

In [ ]:
#### removing the outliers and chacking the data agian
q = df["sum_gamerounds"].quantile(0.999)
df1 = df[df["sum_gamerounds"] < q]
df1.plot.scatter(x='version', y='sum_gamerounds',
                 title= "Scatter plot between two variables X and Y after removing the outliers")
plt.show()

# Furthur exploring the data

In [ ]:

tab = df1.groupby(["version", "retention_1" , "retention_7" ]).sum_gamerounds.agg(["count", "median", "mean", "std", "max"])
print(tab)

In [ ]:
#### separating the groups
versionA = df1[df1["version"]== "gate_30"]
versionB = df1[df1["version"]== "gate_40"]

###looking at 3 conversion metrics for two versions
variables = ['sum_gamerounds', "retention_1" , "retention_7"]

for var in variables:
    print(" ______ A/B Testing for %s " %var)
    print("Histogram by Groups for %s" %var ,"\n")
    sns.distplot( versionA[var] , color="blue", label="version1")
    sns.distplot( versionB[var] , color="red", label="version2")
    plt.legend()
    plt.show()
    
    
        ##### step 1: test for norality of data
    test_statistic_A, shapiro_pvalue_A= shapiro(versionA[var])
    test_statistic_B, shapiro_pvalue_B = shapiro(versionB[var])
    print("test for Normality:")
    print("versionA_shaphiro_p = %s, versionB_shaphiro_p = %s" %(shapiro_pvalue_A, shapiro_pvalue_B), "\n")



    if (shapiro_pvalue_A > 0.05 ) & (shapiro_pvalue_A > 0.05):
        # "H0: Normal Distribution"
        print("We fail to reject the null hypothesis, the distribution is normal ")

        # Parametric Test
        #### test for homogenity
        # Assumption: Homogeneity of variances
        leveneTest = stats.levene(versionA[var], versionB[var])[1]
        # H0: Homogeneity: False
        # H1: Heterogeneous: True

        if leveneTest  > 0.5:
            
            print("We fail to reject the null hypothesis, two distribution are homogeous ")
            # Homogeneity
            ttest = stats.ttest_ind(versionA[var], versionB[var], equal_var=True)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
        else:
            # Heterogeneous
            print("We reject the null hypothesis, two distribution are not homogeous ")
            ttest = stats.ttest_ind(versionA[var], versionB[var], equal_var=False)[1]
            # H0: M1 == M2 - False
            # H1: M1 != M2 - True
    else:
        # Non-Parametric Test
        ttest = stats.mannwhitneyu(versionA[var], versionB[var])
        # H0: M1 == M2 - False
        # H1: M1 != M2 - True

    print("Statistic test = %s" %(ttest[0] ))
    print(("pvalue = %s")  %(ttest[1]))
    pvalue = round(ttest[1], 4)

    #####if pvalue is less than 0.05, there is significant difference between two version of the interface
    if pvalue <= 0.5:
        print("testing %s: The pvalue is %s, therefore there is enough evidence to reject the null hypothesis, which "
              "indicates there is statistically significant difference between "
              "two versions of the product" %(var, pvalue)+"\n")

    else:
        print("testing %s: The pvalue is %s, therefore there is not enough evidence to reject the null hypothesis, which "
              "indicates there is not a statistically significant difference between "
              "two versions of the product" %(var, pvalue))

    print("\n")
    print("\n")
    print("\n")
    print("\n")

**Now it is time to provide some recommendaiotn based on result. Lets look at the data again:**


In [ ]:


table = pd.pivot_table(df1,index=["version" ],values = ["sum_gamerounds", "retention_1", "retention_7"],aggfunc=np.mean )
print(table)

These data provide that version one - gate_30_ has a higher conversion rate in all three categories, and probably will meet the stakeholders' needs better. 